# Creating multi-mission, multi-temporal datacube

In [1]:
import openeo

This notebooks shows how to combine timeseries data from two popular missions, Sentinel-1 and Sentinel-2 in a single datacube for further processing. It can be considered a basic template for many use cases.

The uses precomputed backscatter if available, and falls back to compute backscatter on the fly, which works globally, but also consumes more credits.

We also create 10-daily composites, and apply linear interpolation to avoid gaps. Specific methods may of course require different cloud masking and preprocessing options.

In [2]:
c=openeo.connect("openeo.cloud")
c.authenticate_oidc()

sentinel2 = c.load_collection(
    "SENTINEL2_L2A",
    temporal_extent = ["2022-06-04", "2022-08-04"],
    bands = ["B02", "B03", "B04","SCL"],
    max_cloud_cover=95
)

sentinel2 = sentinel2.process(
            "mask_scl_dilation",
            data=sentinel2,
            scl_band_name="SCL",
            kernel1_size=17, kernel2_size=77,
            mask1_values=[2, 4, 5, 6, 7],
            mask2_values=[3, 8, 9, 10, 11],
            erosion_kernel_size=3)

sentinel2 = sentinel2.aggregate_temporal_period("dekad",reducer="median")\
    .apply_dimension(dimension="t", process="array_interpolate_linear")



Authenticated using refresh token.


Some openEO backends offer precomputed Sentinel-1 backscatter. We [inspect the backend metadata](https://open-eo.github.io/openeo-python-client/data_access.html#data-discovery) to check if such a collection is available, otherwise we start from raw GRD and [compute it on the fly](https://open-eo.github.io/openeo-python-client/cookbook/ard.html#sar-backscatter).



In [3]:
S1_collection = "SENTINEL1_GRD"
if "SENTINEL1_GRD_SIGMA0" in c.list_collection_ids():
    S1_collection = "SENTINEL1_GRD_SIGMA0"

S1_collection

'SENTINEL1_GRD'

In [4]:

sentinel1 = c.load_collection(
    S1_collection,
    temporal_extent = ["2022-06-04", "2022-08-04"],
    bands = ["VV","VH"]
)

if S1_collection == "SENTINEL1_GRD":
    sentinel1 = sentinel1.sar_backscatter(
        coefficient='sigma0-ellipsoid',
        local_incidence_angle=False,
        elevation_model='COPERNICUS_30')

sentinel1 = sentinel1.aggregate_temporal_period("dekad",reducer="median")\
    .apply_dimension(dimension="t", process="array_interpolate_linear")

Now we can simply combine both cubes. Resampling is performed implicitly if needed, but explicit resampling can also be specified.

In [5]:
merged = sentinel2.merge_cubes(sentinel1)

The next block receives the combined Sentinel-1 and Sentinel-2 input, and transforms it using whatever method. 
This can be for instance a neural network based on PyTorch.

This example uses blocks of 128x128 pixels, with an 8 pixel overlap. Sizes for the time and bands dimensions are not specified, which means they will be fully included.

The UDF in this example also shows how to print statements to the logging, this is an easy way to get a better sense of the XArray data that is passed in. More information on UDF's can be found in [the documentation](https://open-eo.github.io/openeo-python-client/udf.html).

In [6]:
my_udf = openeo.UDF("""
from openeo.udf import XarrayDataCube
from openeo.udf.debug import inspect

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    array = cube.get_array()
    inspect(array,level="ERROR",message="inspecting input cube")
    array.values = 0.0001 * array.values
    return cube
""")

fused = merged.apply_neighborhood(my_udf, size=[
        {'dimension': 'x', 'value': 112, 'unit': 'px'},
        {'dimension': 'y', 'value': 112, 'unit': 'px'}
    ], overlap=[
        {'dimension': 'x', 'value': 8, 'unit': 'px'},
        {'dimension': 'y', 'value': 8, 'unit': 'px'}
    ])

In [7]:
spatial_extent = {'west': 4.45, 'east': 4.70, 'south': 51.16, 'north': 51.22, 'crs': 'epsg:4326'}
job=fused.filter_bbox(spatial_extent).execute_batch("result.nc", title="Sentinel composite", filename_prefix="merged_cube")

0:00:00 Job 'j-beec61e8511149d19cc3b6627a19888a': send 'start'
0:00:11 Job 'j-beec61e8511149d19cc3b6627a19888a': created (progress N/A)
0:00:16 Job 'j-beec61e8511149d19cc3b6627a19888a': created (progress N/A)
0:00:23 Job 'j-beec61e8511149d19cc3b6627a19888a': created (progress N/A)
0:00:31 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:00:42 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:00:55 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:01:10 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:01:30 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:01:54 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:02:24 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:03:01 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:03:48 Job 'j-beec61e8511149d19cc3b6627a19888a': running (progress N/A)
0:04:46 Job 'j-beec61e8511149d19cc3b6627a19888a': running (pr

When the job is finished, are downloaded as netCDF and can be inspected using XArray or a desktop viewer like QGis.


In [9]:
import xarray as xr
xr.open_dataset("result.nc")

<xarray.Dataset>
Dimensions:  (t: 7, x: 1762, y: 706)
Coordinates:
  * t        (t) datetime64[ns] 2022-06-01 2022-06-11 ... 2022-07-21 2022-08-01
  * x        (x) float64 6.013e+05 6.013e+05 6.013e+05 ... 6.189e+05 6.189e+05
  * y        (y) float64 5.676e+06 5.676e+06 5.676e+06 ... 5.669e+06 5.669e+06
Data variables:
    crs      |S1 ...
    B02      (t, y, x) float64 ...
    B03      (t, y, x) float64 ...
    B04      (t, y, x) float64 ...
    SCL      (t, y, x) float64 ...
    VV       (t, y, x) float64 ...
    VH       (t, y, x) float64 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.14.1a1
    description:  
    title:

You can also inspect the result in the openEO editor:
![Result in openEO editor](result.png "Batch job result")